In [ ]:
%matplotlib inline
import matplotlib
import seaborn as sns
matplotlib.rcParams['savefig.dpi'] = 144

In [ ]:
import random
import toolz
from pyspark import SparkContext, SparkConf

In [ ]:
from pyspark import SparkContext
sc = SparkContext("local[*]", "pyspark_df")
print sc.version

# Alternatively...
# conf = SparkConf().setAppName("pyspark_df").setMaster("local[*]")
# sc = SparkContext(conf=conf)

In [ ]:
# needed to convert RDDs into DataFrames
from pyspark.sql import SQLContext
from pyspark.sql.functions import UserDefinedFunction as udf
from pyspark.sql.types import DoubleType
sqlContext = SQLContext(sc)

# DataFrames

## Motivation and Spark SQL

Spark SQL is the current effort to provide support for writing SQL queries in Spark. Newer versions support Hive, Parquet, and other data sources. [Docs](http://spark.apache.org/docs/latest/sql-programming-guide.html)

The key feature of Spark SQL is the use of DataFrames instead of RDDs. A DataFrame is a distributed collection of data organized into named columns, and operations on DataFrames are first parsed through an optimized execution engine which streamlines and may even reorder the request to optimize execution. The keyword to search here is Catalyst.

Under the hood, operations on DataFrames are boiled down to operations on RDDs, but the RDDs are created by the execution engine, and not directly by the user. It is also possible to convert RDDs to DataFrames and vice versa.

The Spark ML package, unlike MLlib, uses DataFrames as inputs and outputs.

**Question:** What is an example of a "bad" sequence of operations which should be reordered for optimal performance?

DataFrames are...

* Immutable, like RDDs
* Lineage is remembered, like RDDs (resiliency)
* Lazy execution, like RDDs
* So why do we care?


DataFrames are an abstraction that lets us think of data in a familiar form (Panda, data.frame, SQL table, etc.).

We can use a similar API to RDDs!

Access to SQL-like optimizations and cost analysis due to it being in a columnar format.

What about type safety?

What are these UDF things?

In [ ]:
data = sc.parallelize(xrange(1,10001)) \
         .map(lambda x: (random.random(), random.random()))

In [ ]:
df = data.toDF()
# Note: this isn't always so easy, you may need to explicitly specify a schema

In [ ]:
df.printSchema()

In [ ]:
df = df.withColumnRenamed("_1", "x").withColumnRenamed("_2", "y")
df.write.save("parquet_demo_pyspark", format="parquet")
# Another (older) syntax
# df.write.parquet("file:///home/vagrant/datacourse/module5/demo")

Try rerunning the above cell.

Save modes:
* error
* append
* overwrite
* ignore (ie. CREATE TABLE IF NOT EXISTS)

In [ ]:
df.write.mode("ignore").parquet("parquet_demo_pyspark")

In [ ]:
dfp = sqlContext.read.parquet("parquet_demo_pyspark")

In [ ]:
dfp.describe("x").show()

In [ ]:
filtered_dfp = dfp.filter(dfp["x"] < 0.5)

In [ ]:
filtered_dfp.count()

## Exploring the Catalyst Optimzer

In [ ]:
filtered_dfp.explain(True)

In [ ]:
filtered_df = df.filter(df["x"] < 0.5)

In [ ]:
filtered_df.explain(True)

In [ ]:
filtered_df = df.filter(df["x"] < 0.5).filter(df["y"] < 0.5)

In [ ]:
filtered_df.explain(True)

In [ ]:
filtered_dfp = dfp.filter(dfp["x"] < 0.5).filter(dfp["y"] < 0.5)

In [ ]:
filtered_dfp.explain(True)

Under the hood, it's just manipulating trees based on rules.
The introductory [blog post](https://databricks.com/blog/2015/04/13/deep-dive-into-spark-sqls-catalyst-optimizer.html) has good pictures.

### Project Tungsten

* Memory management and GC (better than the JVM)
* Cache-aware computation
* Codegen (compile queries into Java bytecode)

Cache-aware computation example:
* Case 1: pointer -> key, value
* Case 2: ke, pointer -> key, value

The CPU has to find keys for sort purposes. This helps it find them faster.

[More](https://databricks.com/blog/2015/04/28/project-tungsten-bringing-spark-closer-to-bare-metal.html)

### DataFrame performance and tuning

See [here](http://spark.apache.org/docs/latest/sql-programming-guide.html#performance-tuning) for details.

## SQL and DataFrames

In [ ]:
# Requires Hive to permanently store tables
df.registerTempTable('nums')  # This is NOT the same as a temp table in SQL proper
sql_df = sqlContext.sql("select x, y from nums where y > 0.9 limit 3")
sql_df.show()

In [ ]:
sql_df.explain(True)

*Reminder:* Check the UI (port 4040 by default) for tables in memory.

*Reminder:* A number of interactive tutorials are available on the DataBricks [community cloud](https://community.cloud.databricks.com). I highly recommend making an account and checking out the guide.

This is also a good place to learn about connecting to databases like Cassandra or using JDBC protocol.

## Adding columns and functions

Because DataFrames are immutable, adding new information means appending columns to an existing DataFrame.

In [ ]:
# Currying lets us specify some of a function's arguments and delay specifying the rest until later.

@toolz.curry
def prediction(threshold, val):
    if val > threshold:
        return 1.0
    else:
        return 0.0

In [ ]:
x_labelizer = udf(prediction(0.5), DoubleType())
y_labelizer = udf(prediction(0.9), DoubleType())

In [ ]:
new_df = dfp.withColumn("x_label", x_labelizer("x")).withColumn("y_label", y_labelizer("y"))

In [ ]:
new_df.show()

## Type safety and Datasets

In [ ]:
rdd = new_df.rdd
row = rdd.take(1)
row

In [ ]:
# Remember that take always returns a list of results
print type(row)

In [ ]:
row = row[0]
print type(row)

In Python, we're not too worried about type safety. But it's important to note that in Scala/Java, these Row objects do not contain the type information of the objects inside them and therefore type safety can be lost converting from RDDs to DataFrames. [DataSets](http://spark.apache.org/docs/latest/sql-programming-guide.html#datasets-and-dataframes) (fleshed out in Spark 2.0) are a newer incarnation of DataFrames that add encoding information to preserve that type safety.We can, however, drill into Row objects to extract the information we want.

We can, however, drill into Row objects to extract the information we want.

In [ ]:
row[1]

In [ ]:
row.asDict()

*Copyright &copy; 2016 The Data Incubator.  All rights reserved.*